In [1]:
# Import required packages
from datetime import datetime
import numpy as np
import pandas as pd
import torch

from utils.model import *
from utils.robot import Robot
from utils.settings import config as cfg
from utils.utils import *

In [2]:
panda = Robot(verbose=False)
# data generation
J_tr, P_tr, P_ts, F = load_all_data(panda)
# Build Generative model, NSF
config = {
        'subnet_width': 1024,
        'subnet_num_layers': 3,
        'num_transforms': 9,
        'lr': 2.1e-4,
        'lr_weight_decay': 2.7e-2,
        'decay_step_size': 4e4,
        'gamma': 5e-2,
        'batch_size': 128,
        'num_epochs': 10,
    }
# Neural spline flow (NSF) with 3 sample features and 5 context features
flow, optimizer, scheduler = get_flow_model(
        enable_load_model=cfg.use_pretrained,
        num_transforms=config["num_transforms"],
        subnet_width=config["subnet_width"],
        subnet_num_layers=config["subnet_num_layers"],
        lr=config["lr"],
        lr_weight_decay=config["lr_weight_decay"],
        decay_step_size=config["decay_step_size"],
        gamma=config["gamma"],
        device='cuda')
knn = get_knn(P_tr=P_tr)

F load successfully from /home/luca/ikpflow/data/panda/train/F-2500000-7-3-4.npy
Model load successfully from /home/luca/ikpflow/weights/panda/nsf.pth
knn load successfully from /home/luca/ikpflow/data/panda/train/knn-2500000-7-3-4-normFalse.pickle


In [3]:
mu = torch.zeros(size=(cfg.n,))
NUM_DATA = 100
NUM_SAMPLES = 1000

In [4]:
test(robot=panda, P_ts=P_ts[:NUM_DATA], F=F, solver=flow, knn=knn, K=NUM_SAMPLES, print_report=True)

       position errors (m)
count        100000.000000
mean              0.004769
std               0.006784
min               0.000061
25%               0.002320
50%               0.003715
75%               0.005678
max               0.500852
average inference time (of 100 P): 0.09 sec.


,position errors (m)
0,0.004814
1,0.009589
2,0.004205
3,0.004369
4,0.004447
...,...
99995,0.002365
99996,0.002878
99997,0.005684
99998,0.003119


In [26]:
from jrl.robots import Panda
from jrl.evaluation import pose_errors_cm_deg
import torch

def assert_poses_almost_equal(poses_1, poses_2):
    pos_errors_cm, rot_errors_deg = pose_errors_cm_deg(poses_1, poses_2)
    assert (pos_errors_cm.max().item() < 0.01) and (rot_errors_deg.max().item() < 0.1)

robot = Panda()
joint_angles, poses = robot.sample_joint_angles_and_poses(n=5, return_torch=True) # sample 5 random joint angles and matching poses

# Run forward-kinematics
poses_fk = robot.forward_kinematics_batch(joint_angles) 
assert_poses_almost_equal(poses, poses_fk)

# Run inverse-kinematics
ik_sols = joint_angles + 0.1 * torch.randn_like(joint_angles) 
for i in range(5):
    ik_sols = robot.inverse_kinematics_single_step_levenburg_marquardt(poses, ik_sols)
assert_poses_almost_equal(poses, robot.forward_kinematics_batch(ik_sols))

WorldModel::LoadRobot: /home/luca/.cache/jrl/temp_urdfs/panda_arm_hand_formatted_link_filepaths_absolute.urdf
joint mimic: no multiplier, using default value of 1 
joint mimic: no offset, using default value of 0 
URDFParser: Link size: 17
URDFParser: Joint size: 12
LoadAssimp: Loaded model /home/luca/miniconda3/lib/python3.9/site-packages/jrl/urdfs/panda/meshes/visual/link0.dae (59388 verts, 20478 tris)
LoadAssimp: Loaded model /home/luca/miniconda3/lib/python3.9/site-packages/jrl/urdfs/panda/meshes/visual/link1.dae (37309 verts, 12516 tris)
LoadAssimp: Loaded model /home/luca/miniconda3/lib/python3.9/site-packages/jrl/urdfs/panda/meshes/visual/link2.dae (37892 verts, 12716 tris)
LoadAssimp: Loaded model /home/luca/miniconda3/lib/python3.9/site-packages/jrl/urdfs/panda/meshes/visual/link3.dae (42512 verts, 14233 tris)
LoadAssimp: Loaded model /home/luca/miniconda3/lib/python3.9/site-packages/jrl/urdfs/panda/meshes/visual/link4.dae (43520 verts, 14620 tris)
LoadAssimp: Loaded model /ho

In [27]:
joint_angles

tensor([[-1.0819,  1.6305, -2.4557, -0.1891, -0.1327,  1.6261,  2.0100],
        [ 1.9208, -1.2904,  1.8064, -1.5387, -1.0249,  0.3911,  0.5344],
        [ 1.6244,  1.3597,  2.7328, -2.7020, -0.5981,  0.1660,  0.7866],
        [ 2.2464,  0.6602, -0.6449, -0.7905,  2.8777, -0.0133,  1.9010],
        [-1.0067,  0.8711, -1.5237, -2.4659,  1.7940,  2.8756,  1.8034]],
       device='cuda:0')

In [ ]:
df = pd.DataFrame(data=data, columns=[f'jt_{i}' for i in range(7)] + ['ml'])

In [ ]:
df.boxplot(by='ml', figsize=(10, 10), layout=(5, 2), fontsize=10)

In [ ]:
x_trans = load_numpy(file_path=config.x_trans_train_path)
data = np.column_stack((x_trans, label))
df_trans = pd.DataFrame(data=data, columns=[i for i in range(4)] + ['ml'])

In [ ]:
df_trans.boxplot(by='ml', figsize=(10, 10), layout=(5, 2), fontsize=10)

In [ ]:
for i in range(4):
    print(df_trans.groupby('ml')[i].describe())

In [ ]:
for i in range(7):
    print(df.groupby('ml')[f'jt_{i}'].describe())